In [ ]:
"""PROGRAM TO COMBINE METAGENOMES FROM ALL SAMPLE SITES INTO A SINGLE DATABASE AS A 
LIST OF LISTS.  EACH TAXON HSS A ROW: [ TAXON, SUM OVER ALL SITES, AVERAGE FREQ ] 
ONLY TAXA WITH AVERAGE FREQUENCIES > 10^-5 ARE INCLUDED IN THE OUTPUT FILE.
"""
numprocess = 300
import math
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.font_manager as fp
import random
from google.colab import drive
drive.mount('/content/gdrive')

#1. THE OUTFILE NAME. IF A MG IS TO LEFT LEFT OUT,INSERT IT IN PLACE OF THE "0":
outname = '/content/gdrive/Shared drives/4.Venetian Canals/Joined MG Files/Genus.Frqs.All.MGs.Joined.csv'

#2. PASTE IN THE NAMES OF ALL THE INPUT METAGENOME FILES: 

genus_files = ['...path.../Genus.Site1.Loop1.TRANSPOSED.csv',
         '...path.../Genus.Site2.Loop3.TRANSPOSED.csv',
         '...path.../Genus.Site3.Loop2.TRANSPOSED.csv',
         '...path.../Genus.Site4.Loop1.TRANSPOSED.csv',
         '...path.../Genus.Site5.Loop3.TRANSPOSED.csv',
         '...path.../Genus.Site6.Loop2.TRANSPOSED.csv']

nummetas = len(genus_files)
stringmetas =[]
for x in range(nummetas):   # for reading files and converting each taxon to a list of [taxon, number]
  with open(genus_files[x], 'r') as f:
    data = f.read()
    stringmetas.append(str(data))
  f.close()
print('Number of metagenomes =', len(stringmetas))
print()

cleanmetas =[]            # gets rid of unwanted test, converts number from string to integer
numprint = 4
for x in range(nummetas):
  datastring = stringmetas[x].replace('unclassified (derived from ', '').replace(')', '').replace('Candidatus ', '')
  metalines = datastring.split('\n')
  metalines.pop()
  integermeta =[]
  for y in range(len(metalines)):
    row = metalines[y].split(',')
    integermeta.append([row[0], int(row[1]), 0.0])
  cleanmetas.append(integermeta)

totalslist =[]             # counts the total number of taxa in each metagenome
for x in range(nummetas):
  count =0
  for y in range(len(cleanmetas[x])):
    count += cleanmetas[x][y][1]
  totalslist.append(count)
#print('Total counts for each metagenone: ', totalslist)

for x in range(nummetas):
  for y in range(len(cleanmetas[x])):
    cleanmetas[x][y][2] += cleanmetas[x][y][1] / totalslist[x]

taxonset = set()
for x in range(nummetas):
  for y in range(numprocess):
    if cleanmetas[x][y][0] not in taxonset:
      taxonset.add(cleanmetas[x][y][0])

taxonlist = list(taxonset)
zeros =[]
for x in range(len(genus_files)):
  zeros.append(0.0)
 
cvcm =[]
for x in range(len(taxonlist)):
  row = [ taxonlist[x], 0.0 ] + zeros  
  for y in range(nummetas):
    for z in range(len(cleanmetas[y])):
      if cleanmetas[y][z][0] == taxonlist[x]:
        row[1] += cleanmetas[y][z][2] 
  cvcm.append(row)

cvcm.sort(key = lambda x: x[1])
cvcm.reverse()

for x in range(len(cvcm)):
  for y in range(nummetas):
    for z in range(len(cleanmetas[y])):
      if cleanmetas[y][z][0] == cvcm[x][0]:
        cvcm[x][y+2] = cleanmetas[y][z][2]

print('The top 5 rows:')
for x in range(5):
  print(cvcm[x])
